In [34]:
import pandas as pd
import os
import numpy as np

In [64]:
data = os.path.abspath("data.csv")

df = pd.read_csv(data)
#df['INSTALLATION_YEAR'].head()

In [65]:
df

,case_idx,data_source,case_lat,case_lon,case_date,COMMODITY_RELEASED_TYPE,INTENTIONAL_RELEASE_BBLS,UNINTENTIONAL_RELEASE_BBLS,FATAL,INJURE,...,MOP_PSIG,PIPELINE_FUNCTION,SCADA_IN_PLACE_IND,INVESTIGATION_STATUS,EMPLOYEE_DRUG_TEST_IND,CONTRACTOR_DRUG_TEST_IND,zone,TAVG,inst_age_in_days,accident_pressure_as_%_mop_psig
0,0,gravity,33.198797,-93.420317,2022-12-31,CRUDE OIL,NaN,65.000000,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Hot-Dry,9.360,NaN,NaN
1,1,gravity,31.714380,-101.939180,2022-12-30,CRUDE OIL,NaN,3.400000,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Hot-Dry,NaN,NaN,NaN
2,2,gravity,32.174070,-101.799950,2022-12-26,CRUDE OIL,NaN,5.000000,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Hot-Dry,NaN,NaN,NaN
3,3,gravity,32.444010,-101.585110,2022-12-26,CRUDE OIL,NaN,2.000000,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Very Cold,-9.275,NaN,NaN
4,4,gravity,29.525672,-97.152163,2022-12-25,CRUDE OIL,NaN,0.210000,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Hot-Humid,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8129,8129,gas_gathering,32.316770,-101.555920,2022-08-17,NATURAL GAS,0.000000,471450.834882,0,0,...,50.0,NaN,0.0,"NO, THE FACILITY WAS NOT MONITORED BY A CONTRO...",NaN,NaN,Hot-Dry,29.785,2054.0,90.000000
8130,8130,gas_gathering,36.750726,-107.841400,2022-08-09,NATURAL GAS,0.000000,917788.497222,0,0,...,500.0,NaN,0.0,"NO, THE FACILITY WAS NOT MONITORED BY A CONTRO...",NaN,NaN,Hot-Dry,28.645,2777.0,34.000000
8131,8131,gas_gathering,32.420140,-102.115860,2022-06-06,NATURAL GAS,0.000000,905677.179968,0,0,...,NaN,NaN,1.0,"NO, THE OPERATOR DID NOT FIND THAT AN INVESTIG...",NaN,NaN,Hot-Dry,NaN,8557.0,NaN
8132,8132,gas_gathering,32.369967,-101.386511,2022-05-25,NATURAL GAS,39231.762523,18621.150278,0,0,...,50.0,NaN,0.0,"NO, THE FACILITY WAS NOT MONITORED BY A CONTRO...",NaN,NaN,Cold,7.025,1970.0,90.000000


In [66]:
data_instalation_year = os.path.abspath("df_awarie_2010-2022_p1_ivan.csv")
df_year = pd.read_csv(data_instalation_year)
#df = pd.concat([df, df_year['INSTALLATION_YEAR']], axis=1)


/var/folders/wh/7_ljcysd4cldmfnns18997580000gn/T/ipykernel_4106/1975161469.py:2: DtypeWarning: Columns (19,49,51,52,53,56,57,58,59,67,68,82,84,89,95,97,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,116,138,143,151,152,153,155,156,157,159,160,161,162,171,172,173,176,177,185,188,191,192,193,194,195,197,202,203,204,205,206,207,208,209,210,216,218,220,235,236,237,238,239,240,241,242,243,244,245,249,251,252,257,263,279,280,281,282,283,292) have mixed types. Specify dtype option on import or set low_memory=False.
  df_year = pd.read_csv(data_instalation_year)


In [38]:
#df['INSTALLATION_YEAR'].head()

In [39]:
df_first = df.iloc[:,:20]
df_second = df.iloc[:,23:60]
df_first

,case_idx,data_source,case_lat,case_lon,case_date,COMMODITY_RELEASED_TYPE,INTENTIONAL_RELEASE_BBLS,UNINTENTIONAL_RELEASE_BBLS,FATAL,INJURE,ON_OFF_SHORE,IGNITE_IND,EXPLODE_IND,NUM_PUB_EVACUATED,FEDERAL,LOCATION_TYPE,CROSSING,ITEM_INVOLVED,MATERIAL_INVOLVED,EST_COST_OPER_PAID
0,0,gravity,33.198797,-93.420317,2022-12-31,CRUDE OIL,NaN,65.000000,0,0,ONSHORE,0,NaN,0.0,0.0,PIPELINE RIGHT-OF-WAY,0.0,PIPE,CARBON STEEL,100000.0
1,1,gravity,31.714380,-101.939180,2022-12-30,CRUDE OIL,NaN,3.400000,0,0,ONSHORE,0,NaN,0.0,0.0,TOTALLY CONTAINED ON OPERATOR-CONTROLLED PROPERTY,0.0,OTHER,CARBON STEEL,0.0
2,2,gravity,32.174070,-101.799950,2022-12-26,CRUDE OIL,NaN,5.000000,0,0,ONSHORE,0,NaN,0.0,0.0,TOTALLY CONTAINED ON OPERATOR-CONTROLLED PROPERTY,0.0,OTHER,MATERIAL OTHER THAN CARBON STEEL,0.0
3,3,gravity,32.444010,-101.585110,2022-12-26,CRUDE OIL,NaN,2.000000,0,0,ONSHORE,0,NaN,0.0,0.0,PIPELINE RIGHT-OF-WAY,0.0,OTHER,MATERIAL OTHER THAN CARBON STEEL,0.0
4,4,gravity,29.525672,-97.152163,2022-12-25,CRUDE OIL,NaN,0.210000,0,0,ONSHORE,0,NaN,0.0,0.0,TOTALLY CONTAINED ON OPERATOR-CONTROLLED PROPERTY,0.0,OTHER,CARBON STEEL,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8129,8129,gas_gathering,32.316770,-101.555920,2022-08-17,NATURAL GAS,0.000000,471450.834882,0,0,ONSHORE,1,0.0,0.0,0.0,PIPELINE RIGHT-OF-WAY,0.0,PIPE,PLASTIC,0.0
8130,8130,gas_gathering,36.750726,-107.841400,2022-08-09,NATURAL GAS,0.000000,917788.497222,0,0,ONSHORE,0,NaN,0.0,1.0,OPERATOR-CONTROLLED PROPERTY,0.0,COMPRESSOR,CARBON STEEL,0.0
8131,8131,gas_gathering,32.420140,-102.115860,2022-06-06,NATURAL GAS,0.000000,905677.179968,0,0,ONSHORE,0,NaN,0.0,0.0,PIPELINE RIGHT-OF-WAY,0.0,PIPE,CARBON STEEL,0.0
8132,8132,gas_gathering,32.369967,-101.386511,2022-05-25,NATURAL GAS,39231.762523,18621.150278,0,0,ONSHORE,0,NaN,0.0,0.0,PIPELINE RIGHT-OF-WAY,1.0,PIPE,PLASTIC,0.0


In [40]:
df_second

,EST_COST_ENVIRONMENTAL,EST_COST_OTHER,CAUSE,CAUSE_DETAILS,NARRATIVE,SYSTEM_PART_INVOLVED,INCIDENT_AREA_TYPE,PIPE_FACILITY_TYPE,RELEASE_TYPE,COULD_BE_HCA,MOP_PSIG,PIPELINE_FUNCTION,SCADA_IN_PLACE_IND,INVESTIGATION_STATUS,EMPLOYEE_DRUG_TEST_IND,CONTRACTOR_DRUG_TEST_IND,zone,TAVG,inst_age_in_days,accident_pressure_as_%_mop_psig
0,750000.0,0.0,CORROSION FAILURE,EXTERNAL CORROSION,THE RELEASE OCCURRED NEAR A NORMALLY DRY DRAIN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hot-Dry,9.360,NaN,NaN
1,2800.0,0.0,EQUIPMENT FAILURE,THREADED CONNECTION/COUPLING FAILURE,"ON DECEMBER 30, 2022, A CENTURION PIPELINE, L....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hot-Dry,NaN,NaN,NaN
2,1000.0,0.0,EQUIPMENT FAILURE,NON-THREADED CONNECTION FAILURE,THE RELEASE WAS THE RESULT OF THE FAILURE OF A...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hot-Dry,NaN,NaN,NaN
3,1000.0,0.0,EQUIPMENT FAILURE,OTHER EQUIPMENT FAILURE,DURING ROUTINE LACT SURVEILLANCE TARGA OPERATI...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Very Cold,-9.275,NaN,NaN
4,750.0,0.0,EQUIPMENT FAILURE,"FAILURE OF EQUIPMENT BODY (EXCEPT PUMP), TANK ...","ON DECEMBER 25, 2022, AT APPROXIMATELY 3:11 PM...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hot-Humid,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8129,NaN,0.0,EXCAVATION DAMAGE,EXCAVATION DAMAGE BY THIRD PARTY,"ON 8-17-2022 AT 14:24 CST, AN ENTERPRISE PERSO...","ONSHORE PIPELINE, INCLUDING VALVE SITES",UNDERGROUND,INTRASTATE,OTHER,NaN,50.0,NaN,0.0,"NO, THE FACILITY WAS NOT MONITORED BY A CONTRO...",NaN,NaN,Hot-Dry,29.785,2054.0,90.000000
8130,NaN,0.0,INCORRECT OPERATION,EQUIPMENT NOT INSTALLED PROPERLY,"AN HEC OPERATED COMPRESSOR, THAT IS CONNECTED ...",ONSHORE COMPRESSOR STATION EQUIPMENT AND PIPING,ABOVEGROUND,INTRASTATE,LEAK,NaN,500.0,NaN,0.0,"NO, THE FACILITY WAS NOT MONITORED BY A CONTRO...",NaN,NaN,Hot-Dry,28.645,2777.0,34.000000
8131,NaN,0.0,CORROSION FAILURE,INTERNAL CORROSION,THIRD PARTY NOTIFIED DCP MIDSTREAM LOCAL REPRE...,"ONSHORE PIPELINE, INCLUDING VALVE SITES",UNDERGROUND,INTRASTATE,RUPTURE,NaN,NaN,NaN,1.0,"NO, THE OPERATOR DID NOT FIND THAT AN INVESTIG...",NaN,NaN,Hot-Dry,NaN,8557.0,NaN
8132,NaN,0.0,MATERIAL FAILURE OF PIPE OR WELD,"DESIGN-, CONSTRUCTION-, INSTALLATION-, OR FABR...","ON MAY 25, 2022, AT 20:30 CDST PL CONTROL REQU...","ONSHORE PIPELINE, INCLUDING VALVE SITES",UNDERGROUND,INTRASTATE,OTHER,NaN,50.0,NaN,0.0,"NO, THE FACILITY WAS NOT MONITORED BY A CONTRO...",NaN,NaN,Cold,7.025,1970.0,90.000000


In [67]:
nan_columns = df.columns[df.isna().any()].tolist()

if len(nan_columns) == 0:
        print("Nie ma wartości NaN w żadnej kolumnie.")
else:
    print("Kolumny zawierające wartości NaN:")
    print(nan_columns)

    # Obliczanie procentowego udziału wartości NaN w każdej kolumnie
    for col in nan_columns:
        nan_percentage = (df[col].isna().sum() / len(df)) * 100
        print(f"Procent NaN w kolumnie {col}: {nan_percentage:.2f}%")

Kolumny zawierające wartości NaN:
['COMMODITY_RELEASED_TYPE', 'INTENTIONAL_RELEASE_BBLS', 'UNINTENTIONAL_RELEASE_BBLS', 'ON_OFF_SHORE', 'EXPLODE_IND', 'NUM_PUB_EVACUATED', 'FEDERAL', 'LOCATION_TYPE', 'CROSSING', 'ITEM_INVOLVED', 'MATERIAL_INVOLVED', 'EST_COST_OPER_PAID', 'EST_COST_GAS_RELEASED', 'EST_COST_PROP_DAMAGE', 'EST_COST_EMERGENCY', 'EST_COST_ENVIRONMENTAL', 'EST_COST_OTHER', 'NARRATIVE', 'SYSTEM_PART_INVOLVED', 'INCIDENT_AREA_TYPE', 'PIPE_FACILITY_TYPE', 'RELEASE_TYPE', 'COULD_BE_HCA', 'MOP_PSIG', 'PIPELINE_FUNCTION', 'SCADA_IN_PLACE_IND', 'INVESTIGATION_STATUS', 'EMPLOYEE_DRUG_TEST_IND', 'CONTRACTOR_DRUG_TEST_IND', 'zone', 'TAVG', 'inst_age_in_days', 'accident_pressure_as_%_mop_psig']
Procent NaN w kolumnie COMMODITY_RELEASED_TYPE: 1.19%
Procent NaN w kolumnie INTENTIONAL_RELEASE_BBLS: 58.66%
Procent NaN w kolumnie UNINTENTIONAL_RELEASE_BBLS: 0.92%
Procent NaN w kolumnie ON_OFF_SHORE: 16.65%
Procent NaN w kolumnie EXPLODE_IND: 12.79%
Procent NaN w kolumnie NUM_PUB_EVACUATED: 

In [70]:
print(df['PIPELINE_FUNCTION'].value_counts())

PIPELINE_FUNCTION
> 20% SMYS REGULATED TRANSMISSION           4126
TRANSMISSION SYSTEM                         1372
< 20% SMYS REGULATED TRANSMISSION            637
> 20% SMYS REGULATED GATHERING               103
=< 20% SMYS REGULATED GATHERING               99
TRANSMISSION LINE OF DISTRIBUTION SYSTEM      64
OFFSHORE GATHERING                            64
TRANSMISSION IN STORAGE FIELD                 61
TYPE A GATHERING                              20
TYPE C GATHERING                              19
TYPE B GATHERING                               7
Name: count, dtype: int64


In [42]:
for idx, inst_age in enumerate(df['inst_age_in_days']):
    if pd.isnull(inst_age):  
        install_year = df.at[idx, 'INSTALLATION_YEAR']
        if not pd.isnull(install_year) and str(install_year).isdigit():  
            install_year_str = str(int(install_year))  # Convert to string if it's a digit
            
            # Ensure 'install_year_str' is in the correct format
            install_date = pd.to_datetime(install_year_str, format='%Y')
            
            # Ensure 'case_date' is in the correct format
            case_date = pd.to_datetime(df.at[idx, 'case_date'])
            
            age_in_days = (case_date - install_date).days
            df.at[idx, 'inst_age_in_days'] = age_in_days

df.head()

KeyError: 'INSTALLATION_YEAR'

In [43]:
df = df.drop('INSTALLATION_YEAR', axis=1)


KeyError: "['INSTALLATION_YEAR'] not found in axis"

In [44]:
df.head()

,case_idx,data_source,case_lat,case_lon,case_date,COMMODITY_RELEASED_TYPE,INTENTIONAL_RELEASE_BBLS,UNINTENTIONAL_RELEASE_BBLS,FATAL,INJURE,...,MOP_PSIG,PIPELINE_FUNCTION,SCADA_IN_PLACE_IND,INVESTIGATION_STATUS,EMPLOYEE_DRUG_TEST_IND,CONTRACTOR_DRUG_TEST_IND,zone,TAVG,inst_age_in_days,accident_pressure_as_%_mop_psig
0,0,gravity,33.198797,-93.420317,2022-12-31,CRUDE OIL,NaN,65.00,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Hot-Dry,9.360,NaN,NaN
1,1,gravity,31.714380,-101.939180,2022-12-30,CRUDE OIL,NaN,3.40,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Hot-Dry,NaN,NaN,NaN
2,2,gravity,32.174070,-101.799950,2022-12-26,CRUDE OIL,NaN,5.00,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Hot-Dry,NaN,NaN,NaN
3,3,gravity,32.444010,-101.585110,2022-12-26,CRUDE OIL,NaN,2.00,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Very Cold,-9.275,NaN,NaN
4,4,gravity,29.525672,-97.152163,2022-12-25,CRUDE OIL,NaN,0.21,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Hot-Humid,NaN,NaN,NaN


# One Hot Encode

In [45]:
one_hot_col = ["data_source", "COMMODITY_RELEASED_TYPE", "ON_OFF_SHORE", "LOCATION_TYPE", 
               "ITEM_INVOLVED", "MATERIAL_INVOLVED", "CAUSE", "SYSTEM_PART_INVOLVED", 
               "INCIDENT_AREA_TYPE", "PIPE_FACILITY_TYPE", "PIPELINE_FUNCTION", "INVESTIGATION_STATUS",
               "zone"]

In [46]:
delated = ["CAUSE_DETAILS", "NARRATIVE", ""]

In [47]:
one_hot_encoded = pd.get_dummies(df['data_source'], dummy_na=True)

In [48]:
one_hot_encoded

,gas_distribution,gas_gathering,gas_transmission,gravity,hazardous_liquid,NaN
0,False,False,False,True,False,False
1,False,False,False,True,False,False
2,False,False,False,True,False,False
3,False,False,False,True,False,False
4,False,False,False,True,False,False
...,...,...,...,...,...,...
8129,False,True,False,False,False,False
8130,False,True,False,False,False,False
8131,False,True,False,False,False,False
8132,False,True,False,False,False,False


In [49]:
nan_columns = one_hot_encoded.columns[one_hot_encoded.isna().any()].tolist()

if len(nan_columns) == 0:
        print("Nie ma wartości NaN w żadnej kolumnie.")
else:
    print("Kolumny zawierające wartości NaN:")
    print(nan_columns)

    # Obliczanie procentowego udziału wartości NaN w każdej kolumnie
    for col in nan_columns:
        nan_percentage = (one_hot_encoded[col].isna().sum() / len(one_hot_encoded)) * 100
        print(f"Procent NaN w kolumnie {col}: {nan_percentage:.2f}%")

Nie ma wartości NaN w żadnej kolumnie.


In [50]:
test = 0

for col in one_hot_col:
    unique_features = df[col].nunique()
    test += unique_features
    print(f"Ilość unikalnych cech w kolumnie '{col}': {unique_features}")
print(test - len(one_hot_col))

Ilość unikalnych cech w kolumnie 'data_source': 5
Ilość unikalnych cech w kolumnie 'COMMODITY_RELEASED_TYPE': 10
Ilość unikalnych cech w kolumnie 'ON_OFF_SHORE': 2
Ilość unikalnych cech w kolumnie 'LOCATION_TYPE': 7
Ilość unikalnych cech w kolumnie 'ITEM_INVOLVED': 31
Ilość unikalnych cech w kolumnie 'MATERIAL_INVOLVED': 10
Ilość unikalnych cech w kolumnie 'CAUSE': 10
Ilość unikalnych cech w kolumnie 'SYSTEM_PART_INVOLVED': 21
Ilość unikalnych cech w kolumnie 'INCIDENT_AREA_TYPE': 5
Ilość unikalnych cech w kolumnie 'PIPE_FACILITY_TYPE': 8
Ilość unikalnych cech w kolumnie 'PIPELINE_FUNCTION': 11
Ilość unikalnych cech w kolumnie 'INVESTIGATION_STATUS': 5
Ilość unikalnych cech w kolumnie 'zone': 8
120


In [51]:
for col in one_hot_col:
    one_hot_encoded = pd.get_dummies(df[col])
    if df[col].isna().any():
        df[col + "_nan"] = df[col].isna()
        one_hot_encoded.columns = [col + "_" + str(val) for val in one_hot_encoded.columns]
    else:
        one_hot_encoded.columns = [col + "_" + str(val) for val in one_hot_encoded.columns]

    df = pd.concat([df, one_hot_encoded], axis=1)
    df.drop(col, axis=1, inplace=True)

print("DataFrame po zakodowaniu one-hot:")


DataFrame po zakodowaniu one-hot:


In [52]:
df

,case_idx,case_lat,case_lon,case_date,INTENTIONAL_RELEASE_BBLS,UNINTENTIONAL_RELEASE_BBLS,FATAL,INJURE,IGNITE_IND,EXPLODE_IND,...,"INVESTIGATION_STATUS_YES, SPECIFY INVESTIGATION RESULT(S): (SELECT ALL THAT APPLY)",zone_nan,zone_Cold,zone_Hot-Dry,zone_Hot-Humid,zone_Marine,zone_Mixed-Dry,zone_Mixed-Humid,zone_Subarctic,zone_Very Cold
0,0,33.198797,-93.420317,2022-12-31,NaN,65.000000,0,0,0,NaN,...,False,False,False,True,False,False,False,False,False,False
1,1,31.714380,-101.939180,2022-12-30,NaN,3.400000,0,0,0,NaN,...,False,False,False,True,False,False,False,False,False,False
2,2,32.174070,-101.799950,2022-12-26,NaN,5.000000,0,0,0,NaN,...,False,False,False,True,False,False,False,False,False,False
3,3,32.444010,-101.585110,2022-12-26,NaN,2.000000,0,0,0,NaN,...,False,False,False,False,False,False,False,False,False,True
4,4,29.525672,-97.152163,2022-12-25,NaN,0.210000,0,0,0,NaN,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8129,8129,32.316770,-101.555920,2022-08-17,0.000000,471450.834882,0,0,1,0.0,...,False,False,False,True,False,False,False,False,False,False
8130,8130,36.750726,-107.841400,2022-08-09,0.000000,917788.497222,0,0,0,NaN,...,False,False,False,True,False,False,False,False,False,False
8131,8131,32.420140,-102.115860,2022-06-06,0.000000,905677.179968,0,0,0,NaN,...,False,False,False,True,False,False,False,False,False,False
8132,8132,32.369967,-101.386511,2022-05-25,39231.762523,18621.150278,0,0,0,NaN,...,False,False,True,False,False,False,False,False,False,False


In [1]:
nan_columns = df.columns[df.isna().any()].tolist()

if len(nan_columns) == 0:
        print("Nie ma wartości NaN w żadnej kolumnie.")
else:
    print("Kolumny zawierające wartości NaN:")
    print(nan_columns)

    # Obliczanie procentowego udziału wartości NaN w każdej kolumnie
    for col in nan_columns:
        nan_percentage = (df[col].isna().sum() / len(df)) * 100
        print(f"Procent NaN w kolumnie {col}: {nan_percentage:.2f}%")

NameError: name 'df' is not defined

In [54]:
#df.to_csv('NaN_OneHot/processed_data.csv', index=True, index_label='case_idx')